In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model,preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,mean_squared_error, r2_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data= pd.read_excel('/content/drive/MyDrive/DAC Club/archive (1)/train.xlsx')
data

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3898
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7663
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13883
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6219
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13303
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4108
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4146
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7230
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12649


In [ ]:
data.shape

(10683, 11)

In [ ]:
data.isnull().sum().sum()

2

In [ ]:
data = data.dropna(axis=0)
data

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3898
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7663
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13883
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6219
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13303
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4108
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4146
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7230
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12649


In [ ]:
data.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional Info    object
Price               int64
dtype: object

In [ ]:
data['Source'].value_counts()

Delhi       4536
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: Source, dtype: int64

In [ ]:
data['Destination'].value_counts()

Cochin       4536
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

In [ ]:
data = data.drop(['Source','Destination','Dep_Time','Arrival_Time','Additional Info'],axis=1)

In [ ]:
data.dtypes

Airline            object
Date_of_Journey    object
Route              object
Duration           object
Total_Stops        object
Price               int64
dtype: object

In [ ]:
data['Airline']=LabelEncoder().fit_transform(data['Airline'])
data['Airline'].unique()

array([ 3,  1,  4,  8,  6,  2, 10,  0, 11,  5,  7,  9])

In [ ]:
data['day']=data['Date_of_Journey'].str.split('/').str[0]
data['day'] = data['day'].astype(int)
data['month']=data['Date_of_Journey'].str.split('/').str[1]
data['month'] = data['month'].astype(int)
data['year']=data['Date_of_Journey'].str.split('/').str[2]
data['year'] = data['year'].astype(int)
data = data.drop(['Date_of_Journey'],axis=1)
data.dtypes

Airline         int64
Route          object
Duration       object
Total_Stops    object
Price           int64
day             int64
month           int64
year            int64
dtype: object

In [ ]:
data['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [ ]:
data['Total_Stops'] = data['Total_Stops'].replace('non-stop','0')
data['Total_Stops'] = data['Total_Stops'].replace('1 stop','1')
data['Total_Stops'] = data['Total_Stops'].replace('2 stops','2')
data['Total_Stops'] = data['Total_Stops'].replace('3 stops','3')
data['Total_Stops'] = data['Total_Stops'].replace('4 stops','4')

In [ ]:
data['Total_Stops'].value_counts()

1    5625
0    3491
2    1520
3      45
4       1
Name: Total_Stops, dtype: int64

In [ ]:
data['Total_Stops']=data['Total_Stops'].astype(int)

In [ ]:
data.head

<bound method NDFrame.head of        Airline                  Route Duration  ...  day  month  year
0            3              BLR → DEL   2h 50m  ...   24      3  2019
1            1  CCU → IXR → BBI → BLR   7h 25m  ...    1      5  2019
2            4  DEL → LKO → BOM → COK      19h  ...    9      6  2019
3            3        CCU → NAG → BLR   5h 25m  ...   12      5  2019
4            3        BLR → NAG → DEL   4h 45m  ...    1      3  2019
...        ...                    ...      ...  ...  ...    ...   ...
10678        0              CCU → BLR   2h 30m  ...    9      4  2019
10679        1              CCU → BLR   2h 35m  ...   27      4  2019
10680        4              BLR → DEL       3h  ...   27      4  2019
10681       10              BLR → DEL   2h 40m  ...    1      3  2019
10682        1  DEL → GOI → BOM → COK   8h 20m  ...    9      5  2019

[10682 rows x 8 columns]>

In [ ]:
duration_array=[]
for i in data['Duration']:
  for j in i:
    if j == 'h' : 
      break
  else:
    i = '0h '+i
  duration_array.append(i)
  
data['Duration']= duration_array
data['h'] = data['Duration'].str.split(' ').str[0].str.split('h').str[0]
data['m'] = data['Duration'].str.split(' ').str[1].str.split('m').str[0]
data['m']= data['m'].fillna('0',inplace=False)
data['h']= data['h'].astype(int)*60
data['m']= data['m'].astype(int)
data['duration_mins']=data['h']+data['m']
data['duration_mins']

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Name: duration_mins, Length: 10682, dtype: int64

In [ ]:
data = data.drop(['h','m','Duration'],axis=1)
data

,Airline,Route,Total_Stops,Price,day,month,year,duration_mins
0,3,BLR → DEL,0,3898,24,3,2019,170
1,1,CCU → IXR → BBI → BLR,2,7663,1,5,2019,445
2,4,DEL → LKO → BOM → COK,2,13883,9,6,2019,1140
3,3,CCU → NAG → BLR,1,6219,12,5,2019,325
4,3,BLR → NAG → DEL,1,13303,1,3,2019,285
...,...,...,...,...,...,...,...,...
10678,0,CCU → BLR,0,4108,9,4,2019,150
10679,1,CCU → BLR,0,4146,27,4,2019,155
10680,4,BLR → DEL,0,7230,27,4,2019,180
10681,10,BLR → DEL,0,12649,1,3,2019,160


In [ ]:
data.dtypes

Airline           int64
Route            object
Total_Stops       int64
Price             int64
day               int64
month             int64
year              int64
duration_mins     int64
dtype: object

In [ ]:
#to know different categories of route
for i in range(0,10683):
  print(len(str(data['Route'][i])))
  if i==10:
    break

9
21
21
15
15
9
15
15
15
15
15


In [ ]:
route_array=[]
for i in data['Route']:
  if(len(str(i))==9):
    route_array.append(0)
  elif (len(str(i))==15):
    route_array.append(1)
  else:
    route_array.append(2)

data['intRoute']=route_array
data['intRoute']

0        0
1        2
2        2
3        1
4        1
        ..
10678    0
10679    0
10680    0
10681    0
10682    2
Name: intRoute, Length: 10682, dtype: int64

In [ ]:
data=data.drop(['Route'],axis=1)

In [ ]:
y = data['Price']
x = data.drop(['Price'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(x_train)
mm_scaler.transform(x_test)
pca = PCA()
x_train = pd.DataFrame(pca.fit_transform(x_train))
x_test = pd.DataFrame(pca.transform(x_test))

In [ ]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor.fit(x_train, y_train) 

RandomForestRegressor(random_state=0)

In [ ]:
y_pred_test = regressor.predict(x_test)
y_pred_train = regressor.predict(x_train)
display(regressor.score(x_test,y_test))

0.7157158608854697